In [17]:
import pandas as pd
df = pd.read_csv('../data/dataset_rebalanced.csv')

In [18]:
df = df.replace({'Positive': 0, 'Negative': 1})

In [19]:
X = df.iloc[:,:-1]
y = pd.DataFrame(df['Sentiment'])

In [20]:
X

,ability,able,absolutely,access,according,account,accounts,active,add,added,...,world,worldfirst,worlds,worth,wow,wrong,yeah,year,years,yor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
y

,Sentiment
0,1
1,1
2,1
3,1
4,1
...,...
999,0
1000,0
1001,0
1002,0


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

h = 0.02  # step size in the mesh

names = [
    "Gradient Boosting",
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    GradientBoostingClassifier(),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

def classification_report_csv(report,name):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        print(row_data[0])
        print(row_data[1])
        row['class'] = row_data[0]
        row['name'] = name
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    dataframe.to_csv('classification_report.csv', index = False)


def training(train, name, test, fold_no, model, y_train, y_test):
    x_train = train
    x_test = test
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    plot_confusion_matrix(model, x_train,y_train)
    report = classification_report(y_test, y_pred, output_dict=True)
    #classification_report_csv(report,name)
    df = pd.DataFrame(report).transpose()
    df = pd.DataFrame(df.iloc[2,:]).transpose()
    df2 = pd.DataFrame()
    df2['Accuracy'] = df['precision']
    df2['Algorithm'] = name
    df2['FoldNo'] = fold_no
    df2.to_csv('classification_report.csv', mode='a', header=False,index = False)


for name, clf in zip(names, classifiers):
    print(name)
    skf = StratifiedKFold(n_splits=10)
    fold_no = 1
    for train_index,test_index in skf.split(X, y):
        train = X.iloc[train_index,:]
        test = X.iloc[test_index,:]
        y_train = y.iloc[train_index,:]
        y_test = y.iloc[test_index,:]

        selector = SelectKBest(f_classif, k=4)
        selector.fit(train, y_train)
        scores = -np.log10(selector.pvalues_)
        scores[np.isnan(scores)] = 0
        scores = scores / scores.max()
        train = train.loc[:,train.columns[scores>=0.3]]
        test = test[train.columns.values.tolist()]

        training(train,name, test,fold_no, clf,y_train,y_test)
        fold_no += 1

In [ ]:
import pickle

df = pd.read_csv('../data/dataset_rebalanced.csv')
df = df.replace({'Positive': 0, 'Negative': 1})
X = df.iloc[:, :-1]
y = pd.DataFrame(df['Sentiment'])
model = GaussianProcessClassifier(1.0 * RBF(1.0))

selector = SelectKBest(f_classif, k=4)
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
scores[np.isnan(scores)] = 0
scores = scores / scores.max()
X = X.loc[:,X.columns[scores>=0.3]]
model = model.fit(X, y)
# save the model to disk
filename = 'GaussianProcess.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X, y)
print(result)

In [ ]:
df = pd.read_csv("classification_report.csv",names=["Accuracy","Algorithm","Fold"])

In [ ]:
df

In [ ]:
dfgroup = (df.groupby(['Algorithm'], as_index=False)).mean()

In [ ]:
dfgroup2 = (df.groupby(['Algorithm'], as_index=False)).std()

In [ ]:
dfgroup